In [1]:
!pip install dash pandas plotly openpyxl


In [2]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
from dash import dcc, html
import numpy as np
from sklearn.preprocessing import StandardScaler
from dash import dash_table
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
import plotly.graph_objs as go
from sklearn.metrics import pairwise_distances
from pyclustering.cluster.kmedoids import kmedoids
from scipy.cluster.hierarchy import linkage
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import base64
import io
import io
import base64
from matplotlib import pyplot as plt
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, roc_auc_score


# -------------------------------
# 1. Leer y limpiar datos
# -------------------------------

df = pd.read_excel(
    "/Users/danielayanez/Downloads/resumen por item final.xlsx",
    sheet_name=0,
    dtype={'CLIENTE': str, 'CODIGO': str}
)

df.columns = [
    "anio", "mes", "cliente", "codigo_producto", "nombre_producto",
    "unidad_medida", "cantidad", "valor_unitario", "descuento_total",
    "valor_total", "num_factura", "cc", "cat1", "cat2", "cat3",
    "categoria", "departamento", "municipio"
]
df.drop(columns="cc", inplace=True)

# Normalizar texto
df["mes"] = df["mes"].str.strip().str.capitalize()
df["anio"] = df["anio"].astype(str)
df["mes_anio"] = df["mes"] + " - " + df["anio"]

# Orden personalizado de meses
orden_meses = [
    "Mayo", "Junio", "Julio", "Agosto", "Septiembre", "Octubre",
    "Noviembre", "Diciembre", "Enero", "Febrero", "Marzo", "Abril"
]
df["orden_mes"] = df["mes"].map({mes: i for i, mes in enumerate(orden_meses)})
df["orden_fecha"] = df["anio"].astype(int) * 100 + df["orden_mes"]

# -------------------------------
# 2. Gráficos fijos
# -------------------------------

# Evolución mensual
ventas_mensuales = (
    df.groupby(["mes_anio", "orden_fecha"])["valor_total"]
    .sum()
    .reset_index()
    .sort_values("orden_fecha")
)
fig_linea = px.line(
    ventas_mensuales,
    x="mes_anio",
    y="valor_total",
    markers=True,
    title="Evolución mensual del valor total de ventas",
    labels={"mes_anio": "Mes", "valor_total": "Valor total (COP)"},
)
fig_linea.update_traces(line=dict(color='seagreen'))
fig_linea.update_layout(xaxis_tickangle=-45)
fig_linea.update_yaxes(tickprefix="$", tickformat=",~s")

# Top productos por cantidad y valor
top_productos_total = (
    df.groupby("nombre_producto")["cantidad"]
    .sum()
    .sort_values(ascending=False)
    .head(10)
    .reset_index()
)
top_productos_valor = (
    df.groupby("nombre_producto")["valor_total"]
    .sum()
    .sort_values(ascending=False)
    .head(10)
    .reset_index()
)
fig_cantidad = px.bar(
    top_productos_total, x="cantidad", y="nombre_producto", orientation="h",
    title="Top 10 productos por cantidad vendida",
    labels={"cantidad": "Cantidad total", "nombre_producto": "Producto"},
    color_discrete_sequence=["seagreen"]
)
fig_cantidad.update_layout(yaxis={'categoryorder': 'total ascending'})

fig_valor = px.bar(
    top_productos_valor, x="valor_total", y="nombre_producto", orientation="h",
    title="Top 10 productos por valor total vendido",
    labels={"valor_total": "Valor total (COP)", "nombre_producto": "Producto"},
    color_discrete_sequence=["coral"]
)
fig_valor.update_layout(yaxis={'categoryorder': 'total ascending'})
fig_valor.update_xaxes(tickprefix="$", tickformat=",~s")

# Cantidad total por categoría
ventas_por_categoria = (
    df.groupby("categoria")["cantidad"]
    .sum()
    .sort_values(ascending=False)
    .reset_index()
)
fig_categoria = px.bar(
    ventas_por_categoria,
    x="cantidad",
    y="categoria",
    orientation="h",
    title="Cantidad total vendida por categoría",
    labels={"cantidad": "Cantidad total", "categoria": "Categoría"},
    color_discrete_sequence=["coral"]
)
fig_categoria.update_layout(yaxis={'categoryorder': 'total ascending'})

# Análisis por cliente (excluyendo mostrador)
clientes_reales = df[df["cliente"] != "000022222222"]
top_clientes_cantidad = (
    clientes_reales.groupby("cliente")["cantidad"]
    .sum()
    .sort_values(ascending=False)
    .reset_index()
    .head(10)
)
top_clientes_valor = (
    clientes_reales.groupby("cliente")["valor_total"]
    .sum()
    .sort_values(ascending=False)
    .reset_index()
    .head(10)
)
fig_cliente_cantidad = px.bar(
    top_clientes_cantidad,
    x="cantidad",
    y="cliente",
    orientation="h",
    title="Top 10 clientes por cantidad total comprada",
    labels={"cantidad": "Cantidad", "cliente": "Cliente"},
    color_discrete_sequence=["forestgreen"]
)
fig_cliente_cantidad.update_layout(yaxis={'categoryorder': 'total ascending'})

fig_cliente_valor = px.bar(
    top_clientes_valor,
    x="valor_total",
    y="cliente",
    orientation="h",
    title="Top 10 clientes por valor total comprado",
    labels={"valor_total": "Valor total (COP)", "cliente": "Cliente"},
    color_discrete_sequence=["coral"]
)
fig_cliente_valor.update_layout(yaxis={'categoryorder': 'total ascending'})
fig_cliente_valor.update_xaxes(tickprefix="$", tickformat=",~s")

# -------------------------------
# 3. Lista de combinaciones mes-año válidas
# -------------------------------
meses_anio_unicos = (
    df.groupby(["mes", "anio"])
    .size()
    .reset_index(name="conteo")
    .query("conteo > 0")
    .assign(mes_anio=lambda x: x["mes"] + " - " + x["anio"])
    .sort_values(by=["anio", "mes"], key=lambda col: col.map({m: i for i, m in enumerate(orden_meses)} if col.name == "mes" else col))
)

# Valor total de ventas por municipio (excluyendo mostrador)

ventas_por_municipio = (
    clientes_reales.groupby("municipio")["valor_total"]
    .sum()
    .sort_values(ascending=False)
    .reset_index()
)

fig_municipios = px.bar(
    ventas_por_municipio,
    x="valor_total",
    y="municipio",
    orientation="h",
    title="Valor total de ventas por municipio",
    labels={"valor_total": "Valor total (COP)", "municipio": "Municipio"},
    color_discrete_sequence=["coral"]
)
fig_municipios.update_layout(yaxis={'categoryorder': 'total ascending'})
fig_municipios.update_xaxes(tickprefix="$", tickformat=",~s")


# ----------- Preparación de datos para clustering --------------
# Variables base por cliente
total_gastado = clientes_reales.groupby("cliente")["valor_total"].sum().reset_index(name="total_gastado")
total_compras = clientes_reales.groupby("cliente")["cantidad"].sum().reset_index(name="cantidad_total")
total_facturas = clientes_reales.groupby("cliente")["num_factura"].sum().reset_index(name="total_facturas")
num_productos = clientes_reales.groupby("cliente")["codigo_producto"].nunique().reset_index(name="num_productos_distintos")
num_categorias = clientes_reales.groupby("cliente")["categoria"].nunique().reset_index(name="num_categorias_distintas")

# Unión
df_clientes = total_gastado.copy()
df_clientes = df_clientes.merge(total_compras, on="cliente")
df_clientes = df_clientes.merge(total_facturas, on="cliente")
df_clientes = df_clientes.merge(num_productos, on="cliente")
df_clientes = df_clientes.merge(num_categorias, on="cliente")

# Municipio y departamento más frecuente
municipio_principal = clientes_reales.groupby("cliente")["municipio"].agg(lambda x: x.mode().iloc[0])
departamento_principal = clientes_reales.groupby("cliente")["departamento"].agg(lambda x: x.mode().iloc[0])
df_clientes["municipio_principal"] = df_clientes["cliente"].map(municipio_principal)
df_clientes["departamento_principal"] = df_clientes["cliente"].map(departamento_principal)

# Normalizar variables numéricas
variables_numericas = ['total_gastado', 'cantidad_total', 'total_facturas', 'num_productos_distintos', 'num_categorias_distintas']
scaler = StandardScaler()
X_numericas = scaler.fit_transform(df_clientes[variables_numericas])
df_numericas_norm = pd.DataFrame(X_numericas, columns=variables_numericas)

# One-hot encoding de variables geográficas
X_categoricas = pd.get_dummies(df_clientes[['municipio_principal', 'departamento_principal']], drop_first=True).astype(int)

# Matriz final para clustering
X_final = pd.concat([df_numericas_norm, X_categoricas], axis=1)
X_final.insert(0, 'cliente', df_clientes['cliente'].values)


# -------------------------------
# 4. Inicializar Dash con pestañas
# -------------------------------
app = dash.Dash(__name__)

logo_src = "..."

app.layout = html.Div([
    html.Div([
        html.Img(src=logo_src, style={'height': '100px', 'display': 'block', 'margin': '0 auto'}),
        html.H1("Dashboard de Segmentación y Clasificación de clientes", style={'textAlign': 'left', 'marginTop': '10px','fontFamily': 'sans-serif','color':'#04871c'}),
        html.P("Este dashboard interactivo permite explorar el comportamiento de ventas y clientes mediante visualizaciones dinámicas, segmentación por modelos de clustering (K-Means, jerárquico y PAM) y clasificación de clientes frecuentes. Su objetivo es apoyar la toma de decisiones estratégicas basadas en datos reales y patrones de consumo.",
            style={
                'textAlign': 'justify',
                'fontSize': '16px',
                'marginTop': '10px',
                'marginBottom': '20px',
                'paddingLeft': '20px',
                'paddingRight': '20px',
                'fontFamily': 'sans-serif',
                'color': '#333'
            }
        ),
        html.H4("¿Cómo clasificar y segmentar a los clientes para definir estrategias comerciales más efectivas según su comportamiento de compra?", style={'textAlign': 'left', 'marginTop': '10px','fontFamily': 'sans-serif','color':'#333'})
    ]),

    dcc.Tabs([
        dcc.Tab(label='Visión General', children=[

            html.H2("Evolución de ventas", style={'textAlign': 'center', 'marginTop': '40px','fontFamily': 'sans-serif'}),
            html.Hr(),
            dcc.Graph(figure=fig_linea),
            
            html.Hr(),
            html.H2("Top 10 Productos Más Vendidos", style={'textAlign': 'center', 'marginTop': '40px','fontFamily': 'sans-serif'}),
            html.Hr(),
            html.Div([
                html.Div([dcc.Graph(figure=fig_cantidad)], style={'width': '50%', 'display': 'inline-block'}),
                html.Div([dcc.Graph(figure=fig_valor)], style={'width': '50%', 'display': 'inline-block'}),
            ]),
            
            html.H2("Top productos por mes y año", style={'textAlign': 'center', 'marginTop': '40px','fontFamily': 'sans-serif'}),
            html.Div([
                html.Label("Selecciona una combinación Mes - Año:", style={'fontWeight': 'bold','fontFamily': 'sans-serif', 'fontSize': '18px','color': '#0176cc'}),
                dcc.Dropdown(
                    id='filtro_mes_anio',
                    options=[
                        {'label': fila['mes_anio'], 'value': fila['mes_anio']}
                        for _, fila in meses_anio_unicos.iterrows()
                    ],
                    value=meses_anio_unicos['mes_anio'].iloc[0],
                    clearable=False,
                    style={'width': '50%', 'marginBottom': '20px','fontFamily': 'sans-serif'}
                ),
                dcc.Graph(id='grafico_top_productos_mes')
            ]),
            
            html.Hr(),
            html.H2("Cantidad total vendida por categoría", style={'textAlign': 'center', 'marginTop': '40px','fontFamily': 'sans-serif'}),
            html.Hr(),
            dcc.Graph(figure=fig_categoria),

            html.H2("Cantidad vendida por categoría por mes", style={'textAlign': 'center', 'marginTop': '40px','fontFamily': 'sans-serif'}),
            html.Div([
                html.Label("Selecciona una combinación Mes - Año:", style={'fontWeight': 'bold','fontFamily': 'sans-serif', 'fontSize': '18px','color': '#0176cc'}),
                dcc.Dropdown(
                    id='filtro_categoria_mes_anio',
                    options=[
                        {'label': fila['mes_anio'], 'value': fila['mes_anio']}
                        for _, fila in meses_anio_unicos.iterrows()
                    ],
                    value=meses_anio_unicos['mes_anio'].iloc[0],
                    clearable=False,
                    style={'width': '50%', 'marginBottom': '20px','fontFamily': 'sans-serif'}
                ),
                dcc.Graph(id='grafico_categoria_mes')
            ]),
            
            html.Hr(),
            html.H2("Análisis por Cliente", style={'textAlign': 'center', 'marginTop': '40px','fontFamily': 'sans-serif'}),
            html.Hr(),
            html.Div([
                html.Div([dcc.Graph(figure=fig_cliente_cantidad)], style={'width': '50%', 'display': 'inline-block'}),
                html.Div([dcc.Graph(figure=fig_cliente_valor)], style={'width': '50%', 'display': 'inline-block'}),
            ]),

            html.Hr(),
            html.H2("Valor total de ventas por municipio (excluyendo mostrador)", style={'textAlign': 'center', 'marginTop': '40px','fontFamily': 'sans-serif'}),
            html.Hr(),
            dcc.Graph(figure=fig_municipios),
        ]),
        
        dcc.Tab(label='Segmentación de Clientes', children=[
            html.H2("Segmentación de Clientes con Modelos (K-Means , Jerarquico, PAM)", style={'textAlign': 'center', 'marginTop': '40px','fontFamily': 'sans-serif'}),
            html.Hr(),
            html.H3("Segmentación con K-Means", style={'textAlign': 'center', 'marginTop': '20px','fontFamily': 'sans-serif'}),
            html.Hr(),
            html.Div([
                html.Label("Selecciona el número de clústeres (k):", style={'fontWeight': 'bold',}),
                dcc.Slider(
                    id='slider_k',
                    min=2,
                    max=10,
                    step=1,
                    value=4,
                    marks={i: str(i) for i in range(2, 11)},
                    tooltip={"placement": "bottom", "always_visible": True}
                )
            ], style={'marginBottom': '5px'}),
            html.P("* Desplazate en este filtro para selecciónar el número de clústers que deseas evaluar en K-means (K)",
            style={
                'textAlign': 'justify',
                'fontSize': '16px',
                'marginTop': '10px',
                'marginBottom': '20px',
                'paddingLeft': '20px',
                'paddingRight': '20px',
                'fontFamily': 'sans-serif',
                'color': '#333'
            }
        ),
            html.Div([
                # Segmentación con K-Means
                html.Div(id='contenedor_segmentacion'),

                
                html.Div([
                html.Hr(),
                html.H3("Segmentación con Clustering Jerárquico", style={'textAlign': 'center', 'marginTop': '20px','fontFamily': 'sans-serif'}),
                html.Hr(),
                    
                html.Div([
                    html.Label("Selecciona el número de clústeres (k):", style={'fontWeight': 'bold'}),
                    dcc.Slider(
                        id='slider_k_jerarquico',
                        min=2,
                        max=10,
                        step=1,
                        value=3,
                        marks={i: str(i) for i in range(2, 11)},
                        tooltip={"placement": "bottom", "always_visible": True})
                ], style={'marginBottom': '5px'}),
                html.P("* Desplazate en este filtro para selecciónar el número de clústers que deseas evaluar en Clustering Jerárquico",
            style={
                'textAlign': 'justify',
                'fontSize': '16px',
                'marginTop': '10px',
                'marginBottom': '20px',
                'paddingLeft': '20px',
                'paddingRight': '20px',
                'fontFamily': 'sans-serif',
                'color': '#333'
            }
        ),
                html.Div(id='contenedor_segmentacion_jerarquico')
            ]),
                
                html.Hr(),
                html.H3("Segmentación con PAM (K-Medoids)",  style={'textAlign': 'center', 'marginTop': '20px','fontFamily': 'sans-serif'}),
                html.Hr(),
                html.Label("Selecciona el número de clústeres (k):", style={'fontWeight': 'bold'}),
                dcc.Slider(
                    id='slider_k_pam',
                    min=2,
                    max=10,
                    step=1,
                    value=3,
                    marks={i: str(i) for i in range(2, 11)},
                    tooltip={"placement": "bottom", "always_visible": True}
                ),
                html.P("* Desplazate en este filtro para selecciónar el número de clústers que deseas evaluar en PAM (K-Medoids)",
            style={
                'textAlign': 'justify',
                'fontSize': '16px',
                'marginTop': '10px',
                'marginBottom': '20px',
                'paddingLeft': '20px',
                'paddingRight': '20px',
                'fontFamily': 'sans-serif',
                'color': '#333'
            }
        ),
                html.Div(id='contenedor_segmentacion_pam', style={'marginTop': '40px','fontFamily': 'sans-serif'}),

                html.Hr(),
                html.H3("Comparación de Modelos de Clustering", style={'textAlign': 'center', 'marginTop': '20px','fontFamily': 'sans-serif'}),
                html.Hr(),
                html.Div(id='comparacion_modelos', style={'marginBottom': '70px'})
            ])
            
        ]),

        dcc.Tab(label='Clasificación de Clientes', children=[
            html.H2("Clasificación de Clientes Frecuentes", style={'textAlign': 'center', 'marginTop': '40px','fontFamily': 'sans-serif'}),
            html.Hr(),
            html.H3("Regresión Logística", style={'textAlign': 'center', 'marginTop': '20px','fontFamily': 'sans-serif'}),
            html.Hr(),

            html.Div([
                html.Label("Selecciona el número mínimo de meses para que un cliente sea considerado frecuente:",
                           style={'fontWeight': 'bold'}),
                dcc.Slider(
                    id='slider_frecuencia',
                    min=1,
                    max=12,
                    step=1,
                    value=3,  # valor por defecto
                    marks={i: str(i) for i in range(1, 13)},
                    tooltip={"placement": "bottom", "always_visible": True},
                    updatemode='drag'
                )
            ], style={'margin': '30px'}),
            
            html.Div(id='salida_modelo_clasificacion'),

            html.Hr(),
            html.H3("Árbol de Decisión", style={'textAlign': 'center', 'marginTop': '20px','fontFamily': 'sans-serif'}),
            html.Hr(),
            html.Div(id='salida_modelo_arbol'),

            html.Hr(),
            html.H3("Comparación de Modelos de Clasificación", style={'textAlign': 'center', 'marginTop': '20px','fontFamily': 'sans-serif'}),
            html.Hr(),
            html.Div(id='comparacion_modelos_clasificacion', style={'marginBottom': '70px'})

        ])
    ])
])

# -------------------------------
# 5. Callback del gráfico filtrado
# -------------------------------
@app.callback(
    Output('grafico_top_productos_mes', 'figure'),
    Input('filtro_mes_anio', 'value')
)
def actualizar_top_productos(mes_anio_seleccionado):
    mes, anio = mes_anio_seleccionado.split(" - ")
    df_filtrado = df[(df["mes"] == mes) & (df["anio"] == anio)]

    if df_filtrado.empty:
        return px.bar(title=f"No hay datos para {mes_anio_seleccionado}")

    top_productos = (
        df_filtrado.groupby("nombre_producto")["cantidad"]
        .sum()
        .sort_values(ascending=False)
        .head(10)
        .reset_index()
    )

    fig = px.bar(
        top_productos,
        x="cantidad",
        y="nombre_producto",
        orientation="h",
        title=f"Top 10 productos en {mes_anio_seleccionado}",
        labels={"cantidad": "Cantidad", "nombre_producto": "Producto"},
        color_discrete_sequence=["forestgreen"]
    )
    fig.update_layout(yaxis={'categoryorder': 'total ascending'})
    return fig

@app.callback(
    Output('grafico_categoria_mes', 'figure'),
    Input('filtro_categoria_mes_anio', 'value')
)
def actualizar_categoria_mes(mes_anio_seleccionado):
    mes, anio = mes_anio_seleccionado.split(" - ")
    df_filtrado = df[(df["mes"] == mes) & (df["anio"] == anio)]

    if df_filtrado.empty:
        return px.bar(title=f"No hay datos para {mes_anio_seleccionado}")

    ventas_categoria = (
        df_filtrado.groupby("categoria")["cantidad"]
        .sum()
        .sort_values(ascending=False)
        .reset_index()
    )

    fig = px.bar(
        ventas_categoria,
        x="cantidad",
        y="categoria",
        orientation="h",
        title=f"Cantidad vendida por categoría en {mes_anio_seleccionado}",
        labels={"cantidad": "Cantidad", "categoria": "Categoría"},
        color_discrete_sequence=["seagreen"]
    )
    fig.update_layout(yaxis={'categoryorder': 'total ascending'})
    return fig


@app.callback(
    Output('contenedor_segmentacion', 'children'),
    Input('slider_k', 'value')
)
def actualizar_segmentacion(k):
    # Clustering
        # Recalcular X_final desde cero para evitar acumulación de etiquetas
    df_temp = df[df["cliente"] != "000022222222"]

    total_gastado = df_temp.groupby("cliente")["valor_total"].sum().reset_index(name="total_gastado")
    total_compras = df_temp.groupby("cliente")["cantidad"].sum().reset_index(name="cantidad_total")
    total_facturas = df_temp.groupby("cliente")["num_factura"].sum().reset_index(name="total_facturas")
    num_productos = df_temp.groupby("cliente")["codigo_producto"].nunique().reset_index(name="num_productos_distintos")
    num_categorias = df_temp.groupby("cliente")["categoria"].nunique().reset_index(name="num_categorias_distintas")

    df_clientes_kmeans = total_gastado.merge(total_compras, on="cliente") \
                                      .merge(total_facturas, on="cliente") \
                                      .merge(num_productos, on="cliente") \
                                      .merge(num_categorias, on="cliente")

    municipio_principal = df_temp.groupby("cliente")["municipio"].agg(lambda x: x.mode().iloc[0])
    departamento_principal = df_temp.groupby("cliente")["departamento"].agg(lambda x: x.mode().iloc[0])
    df_clientes_kmeans["municipio_principal"] = df_clientes_kmeans["cliente"].map(municipio_principal)
    df_clientes_kmeans["departamento_principal"] = df_clientes_kmeans["cliente"].map(departamento_principal)

    variables_numericas = ['total_gastado', 'cantidad_total', 'total_facturas', 'num_productos_distintos', 'num_categorias_distintas']
    scaler = StandardScaler()
    X_numericas = scaler.fit_transform(df_clientes_kmeans[variables_numericas])
    df_numericas_norm = pd.DataFrame(X_numericas, columns=variables_numericas)

    X_categoricas = pd.get_dummies(df_clientes_kmeans[['municipio_principal', 'departamento_principal']], drop_first=True).astype(int)
    X = pd.concat([df_numericas_norm, X_categoricas], axis=1)
    
    # X = X_final.drop(columns=['cliente'])
    X = X[sorted(X.columns)]  # asegurar mismo orden
    modelo = KMeans(n_clusters=k, random_state=42, n_init=10)
    etiquetas = modelo.fit_predict(X)
    df_resultado = df_clientes.copy()
    df_resultado['cluster_kmeans'] = etiquetas

    # PCA para visualización
    X_pca = PCA(n_components=2).fit_transform(X)
    df_pca = pd.DataFrame(X_pca, columns=['PC1', 'PC2'])
    df_pca['cluster_kmeans'] = etiquetas
    df_pca['cliente'] = df_resultado['cliente']

    # Gráfico PCA
    fig_pca = px.scatter(
        df_pca, x='PC1', y='PC2', color='cluster_kmeans',
        hover_data=['cliente'], title="Distribución de Clústeres con PCA (2D)",
        color_continuous_scale='Sunset'
    )

    # Métricas: inercia y silhouette
    silhouette = silhouette_score(X, etiquetas)
    inercia = modelo.inertia_

    fig_metricas = go.Figure()
    fig_metricas.add_trace(go.Indicator(
        mode="number+delta",
        value=silhouette,
        number={"valueformat": ".3f"},
        title={"text": "Silhouette Score"},
        domain={'x': [0, 0.5], 'y': [0, 1]}
    ))
    fig_metricas.add_trace(go.Indicator(
        mode="number+delta",
        value=inercia,
        number={"valueformat": ".0f"},
        title={"text": "Inercia (Codo)"},
        domain={'x': [0.5, 1], 'y': [0, 1]}
    ))
    fig_metricas.update_layout(title_text="Métricas del Modelo", height=250)

    # Resumen por clúster
    df_resultado = df_clientes.copy()
    df_resultado['cluster_kmeans'] = etiquetas
    resumen = df_resultado.groupby('cluster_kmeans').agg({
        'total_gastado': 'mean',
        'cantidad_total': 'mean',
        'total_facturas': 'mean',
        'num_productos_distintos': 'mean',
        'num_categorias_distintas': 'mean'
    }).round(2).reset_index()

    # Conteo de clientes por clúster
    conteo_kmeans = df_resultado['cluster_kmeans'].value_counts().sort_index()
    conteo_kmeans_html = html.Ul([
        html.Li(f"Clúster {i}: {conteo} clientes") for i, conteo in conteo_kmeans.items()
    ])


    tabla_resumen = dash_table.DataTable(
        columns=[{"name": col, "id": col} for col in resumen.columns],
        data=resumen.to_dict("records"),
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'center'},
        style_header={'fontWeight': 'bold', 'backgroundColor': '#e1e1e1'}
    )

    return [
        html.H3("Métricas del Modelo", style={'marginTop': '20px'}),
        dcc.Graph(figure=fig_metricas),

        html.H3("Visualización de Clústeres (PCA)", style={'marginTop': '40px'}),
        dcc.Graph(figure=fig_pca),

        html.H3("Resumen por Clúster", style={'marginTop': '40px'}),
        tabla_resumen,
        
        html.H3("Cantidad de Clientes por Clúster", style={'marginTop': '20px'}),
        conteo_kmeans_html,

    ]

@app.callback(
    Output('contenedor_segmentacion_jerarquico', 'children'),
    Input('slider_k_jerarquico', 'value')
)
def actualizar_segmentacion_jerarquico(k):
    from sklearn.cluster import AgglomerativeClustering

    df_temp = df[df["cliente"] != "000022222222"]

    # Variables base por cliente
    total_gastado = df_temp.groupby("cliente")["valor_total"].sum().reset_index(name="total_gastado")
    total_compras = df_temp.groupby("cliente")["cantidad"].sum().reset_index(name="cantidad_total")
    total_facturas = df_temp.groupby("cliente")["num_factura"].sum().reset_index(name="total_facturas")
    num_productos = df_temp.groupby("cliente")["codigo_producto"].nunique().reset_index(name="num_productos_distintos")
    num_categorias = df_temp.groupby("cliente")["categoria"].nunique().reset_index(name="num_categorias_distintas")

    # Fusionar los datos
    df_clientes_tmp = total_gastado.merge(total_compras, on="cliente") \
                                   .merge(total_facturas, on="cliente") \
                                   .merge(num_productos, on="cliente") \
                                   .merge(num_categorias, on="cliente")

    # Agregar municipio y departamento
    municipio_principal = df_temp.groupby("cliente")["municipio"].agg(lambda x: x.mode().iloc[0])
    departamento_principal = df_temp.groupby("cliente")["departamento"].agg(lambda x: x.mode().iloc[0])
    df_clientes_tmp["municipio_principal"] = df_clientes_tmp["cliente"].map(municipio_principal)
    df_clientes_tmp["departamento_principal"] = df_clientes_tmp["cliente"].map(departamento_principal)

    # Normalización de las variables numéricas
    variables_numericas = ['total_gastado', 'cantidad_total', 'total_facturas', 'num_productos_distintos', 'num_categorias_distintas']
    scaler = StandardScaler()
    X_numericas = scaler.fit_transform(df_clientes_tmp[variables_numericas])
    df_numericas_norm = pd.DataFrame(X_numericas, columns=variables_numericas)

    # One-Hot Encoding para variables categóricas
    X_categoricas = pd.get_dummies(df_clientes_tmp[['municipio_principal', 'departamento_principal']], drop_first=True).astype(int)

    # Crear la matriz final para clustering
    X = pd.concat([df_numericas_norm, X_categoricas], axis=1)
    X = X[sorted(X.columns)]  # Aseguramos que las columnas estén en orden
    
    modelo = AgglomerativeClustering(n_clusters=k, linkage='ward')
    etiquetas = modelo.fit_predict(X)
    #df_clientes_tmp = df_clientes.copy()
    #df_clientes_tmp['cluster_kmeans'] = etiquetas

    # Calcular la matriz de distancias
    distance_matrix = pairwise_distances(X, metric='euclidean')

    # Calcular el Silhouette Score
    silhouette = silhouette_score(distance_matrix, etiquetas, metric='precomputed')


    # PCA
    # PCA para visualización
    X_pca = PCA(n_components=2).fit_transform(X)
    df_pca = pd.DataFrame(X_pca, columns=["PC1", "PC2"])
    df_pca["cluster_jerarquico"] = etiquetas
    df_pca["cliente"] = df_clientes_tmp["cliente"]

    # Gráfico PCA
    fig_pca = px.scatter(
    df_pca, x="PC1", y="PC2", color="cluster_jerarquico",
    hover_data=["cliente"], title="Distribución de Clústeres Jerárquico con PCA",
    color_continuous_scale="Sunset"
    )

    fig_silhouette = go.Figure()
    fig_silhouette.add_trace(go.Indicator(
        mode="number",
        value=silhouette,
        number={"valueformat": ".3f"},
        title={"text": "Silhouette Score"},
        domain={'x': [0, 1], 'y': [0, 1]}
    ))
    fig_silhouette.update_layout(height=250)

    df_resultado = df_clientes.copy()
    df_resultado['cluster_jerarquico'] = etiquetas
    resumen = df_resultado.groupby('cluster_jerarquico').agg({
        'total_gastado': 'mean',
        'cantidad_total': 'mean',
        'total_facturas': 'mean',
        'num_productos_distintos': 'mean',
        'num_categorias_distintas': 'mean'
    }).round(2).reset_index()

    tabla_resumen = dash_table.DataTable(
        columns=[{"name": col, "id": col} for col in resumen.columns],
        data=resumen.to_dict("records"),
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'center'},
        style_header={'fontWeight': 'bold', 'backgroundColor': '#e1e1e1'}
    )

    # Conteo
    conteo = df_resultado['cluster_jerarquico'].value_counts().sort_index()
    texto_conteo = [
        html.H4("Cantidad de Clientes por Clúster", style={'marginTop': '20px'})
    ] + [html.Li(f"Clúster {i}: {v} clientes") for i, v in conteo.items()]

    return [
        html.H3("Métrica del Modelo (Silhouette)", style={'marginTop': '20px'}),
        dcc.Graph(figure=fig_silhouette),

        html.H3("Visualización de Clústeres con PCA", style={'marginTop': '40px'}),
        dcc.Graph(figure=fig_pca),

        html.H3("Resumen por Clúster", style={'marginTop': '40px'}),
        tabla_resumen,

        html.Div(texto_conteo)
    ]
    
@app.callback(
    Output('contenedor_segmentacion_pam', 'children'),
    Input('slider_k_pam', 'value')
)
def actualizar_segmentacion_pam(k):
# Asegurémonos de tener los datos preprocesados y normalizados
    df_temp = df[df["cliente"] != "000022222222"]

    # Variables base por cliente
    total_gastado = df_temp.groupby("cliente")["valor_total"].sum().reset_index(name="total_gastado")
    total_compras = df_temp.groupby("cliente")["cantidad"].sum().reset_index(name="cantidad_total")
    total_facturas = df_temp.groupby("cliente")["num_factura"].sum().reset_index(name="total_facturas")
    num_productos = df_temp.groupby("cliente")["codigo_producto"].nunique().reset_index(name="num_productos_distintos")
    num_categorias = df_temp.groupby("cliente")["categoria"].nunique().reset_index(name="num_categorias_distintas")

    # Fusionar los datos
    df_clientes_pam = total_gastado.merge(total_compras, on="cliente") \
                                   .merge(total_facturas, on="cliente") \
                                   .merge(num_productos, on="cliente") \
                                   .merge(num_categorias, on="cliente")

    # Agregar municipio y departamento
    municipio_principal = df_temp.groupby("cliente")["municipio"].agg(lambda x: x.mode().iloc[0])
    departamento_principal = df_temp.groupby("cliente")["departamento"].agg(lambda x: x.mode().iloc[0])
    df_clientes_pam["municipio_principal"] = df_clientes_pam["cliente"].map(municipio_principal)
    df_clientes_pam["departamento_principal"] = df_clientes_pam["cliente"].map(departamento_principal)

    # Normalización de las variables numéricas
    variables_numericas = ['total_gastado', 'cantidad_total', 'total_facturas', 'num_productos_distintos', 'num_categorias_distintas']
    scaler = StandardScaler()
    X_numericas = scaler.fit_transform(df_clientes_pam[variables_numericas])
    df_numericas_norm = pd.DataFrame(X_numericas, columns=variables_numericas)

    # One-Hot Encoding para variables categóricas
    X_categoricas = pd.get_dummies(df_clientes_pam[['municipio_principal', 'departamento_principal']], drop_first=True).astype(int)

    # Crear la matriz final para clustering
    X = pd.concat([df_numericas_norm, X_categoricas], axis=1)
    X = X[sorted(X.columns)]  # Aseguramos que las columnas estén en orden
    X_array = X.to_numpy()
    distance_matrix = pairwise_distances(X_array, metric='euclidean')
    
    np.random.seed(42)
    initial_medoids = np.random.choice(len(X_array), size=k, replace=False).tolist()
    pam_instance = kmedoids(data=distance_matrix, initial_index_medoids=initial_medoids, data_type='distance_matrix')
    pam_instance.process()
    clusters = pam_instance.get_clusters()

    # Asignar etiquetas
    labels = np.zeros(len(X_array), dtype=int)
    for cluster_id, indices in enumerate(clusters):
        for index in indices:
            labels[index] = cluster_id
    X_final["cluster_pam"] = labels

    # Silhouette
    silhouette = silhouette_score(distance_matrix, labels, metric='precomputed')

    # PCA
    X_pca = PCA(n_components=2).fit_transform(X_array)
    df_pca = pd.DataFrame(X_pca, columns=["PC1", "PC2"])
    df_pca["cluster_pam"] = labels
    df_pca["cliente"] = X_final["cliente"]

    fig_pca = px.scatter(
        df_pca, x="PC1", y="PC2", color="cluster_pam",
        hover_data=["cliente"], title="Distribución de Clústeres PAM con PCA",
        color_continuous_scale="Sunset"
    )

    fig_silhouette = go.Figure()
    fig_silhouette.add_trace(go.Indicator(
        mode="number",
        value=silhouette,
        number={"valueformat": ".3f"},
        title={"text": "Silhouette Score"},
        domain={'x': [0, 1], 'y': [0, 1]}
    ))
    fig_silhouette.update_layout(height=250)

    # Tabla resumen
    df_resultado = df_clientes.copy()
    df_resultado['cluster_pam'] = labels
    resumen = df_resultado.groupby('cluster_pam').agg({
        'total_gastado': 'mean',
        'cantidad_total': 'mean',
        'total_facturas': 'mean',
        'num_productos_distintos': 'mean',
        'num_categorias_distintas': 'mean'
    }).round(2).reset_index()

    # Conteo de clientes por clúster
    conteo_pam = df_resultado['cluster_pam'].value_counts().sort_index()
    conteo_pam_html = html.Ul([
        html.Li(f"Clúster {i}: {conteo} clientes") for i, conteo in conteo_pam.items()
    ])


    tabla_resumen = dash_table.DataTable(
        columns=[{"name": col, "id": col} for col in resumen.columns],
        data=resumen.to_dict("records"),
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'center'},
        style_header={'fontWeight': 'bold', 'backgroundColor': '#e1e1e1'}
    )

    return [
        html.H3("Métrica del Modelo (Silhouette)", style={'marginTop': '20px'}),
        dcc.Graph(figure=fig_silhouette),

        html.H3("Visualización de Clústeres con PCA", style={'marginTop': '40px'}),
        dcc.Graph(figure=fig_pca),

        html.H3("Resumen por Clúster", style={'marginTop': '40px'}),
        tabla_resumen,

        html.H3("Cantidad de Clientes por Clúster", style={'marginTop': '20px'}),
        conteo_pam_html,
    ]

from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score

@app.callback(
    Output('comparacion_modelos', 'children'),
    Input('slider_k', 'value'),
    Input('slider_k_jerarquico', 'value'),
    Input('slider_k_pam', 'value')
)
def actualizar_comparacion_modelos(k_kmeans, k_jerarquico, k_pam):
    # ------ Recalcular etiquetas de clustering para cada modelo ------
    
    # 1. Datos base
    df_temp = df[df["cliente"] != "000022222222"]
    total_gastado = df_temp.groupby("cliente")["valor_total"].sum().reset_index(name="total_gastado")
    total_compras = df_temp.groupby("cliente")["cantidad"].sum().reset_index(name="cantidad_total")
    total_facturas = df_temp.groupby("cliente")["num_factura"].sum().reset_index(name="total_facturas")
    num_productos = df_temp.groupby("cliente")["codigo_producto"].nunique().reset_index(name="num_productos_distintos")
    num_categorias = df_temp.groupby("cliente")["categoria"].nunique().reset_index(name="num_categorias_distintas")

    df_clientes_base = total_gastado.merge(total_compras, on="cliente") \
                                    .merge(total_facturas, on="cliente") \
                                    .merge(num_productos, on="cliente") \
                                    .merge(num_categorias, on="cliente")

    municipio_principal = df_temp.groupby("cliente")["municipio"].agg(lambda x: x.mode().iloc[0])
    departamento_principal = df_temp.groupby("cliente")["departamento"].agg(lambda x: x.mode().iloc[0])
    df_clientes_base["municipio_principal"] = df_clientes_base["cliente"].map(municipio_principal)
    df_clientes_base["departamento_principal"] = df_clientes_base["cliente"].map(departamento_principal)

    # Escalar + Dummies
    variables_numericas = ['total_gastado', 'cantidad_total', 'total_facturas', 'num_productos_distintos', 'num_categorias_distintas']
    scaler = StandardScaler()
    X_num = scaler.fit_transform(df_clientes_base[variables_numericas])
    df_num = pd.DataFrame(X_num, columns=variables_numericas)
    X_cat = pd.get_dummies(df_clientes_base[['municipio_principal', 'departamento_principal']], drop_first=True).astype(int)
    X = pd.concat([df_num, X_cat], axis=1)
    X = X[sorted(X.columns)]
    X_array = X.to_numpy()

    resultados = []

    # K-Means
    modelo_kmeans = KMeans(n_clusters=k_kmeans, random_state=42, n_init=10)
    etiquetas_kmeans = modelo_kmeans.fit_predict(X)
    silhouette_kmeans = silhouette_score(X, etiquetas_kmeans)
    ch_kmeans = calinski_harabasz_score(X, etiquetas_kmeans)
    db_kmeans = davies_bouldin_score(X, etiquetas_kmeans)
    resultados.append({
        "Modelo": f"K-Means (k={k_kmeans})",
        "Silhouette Score": silhouette_kmeans,
        "Calinski-Harabasz": ch_kmeans,
        "Davies-Bouldin": db_kmeans
    })

    # Jerárquico
    modelo_jer = AgglomerativeClustering(n_clusters=k_jerarquico, linkage='ward')
    etiquetas_jer = modelo_jer.fit_predict(X)
    dist_matrix = pairwise_distances(X, metric='euclidean')
    silhouette_jer = silhouette_score(dist_matrix, etiquetas_jer, metric='precomputed')
    ch_jer = calinski_harabasz_score(X, etiquetas_jer)
    db_jer = davies_bouldin_score(X, etiquetas_jer)
    resultados.append({
        "Modelo": f"Jerárquico (k={k_jerarquico})",
        "Silhouette Score": silhouette_jer,
        "Calinski-Harabasz": ch_jer,
        "Davies-Bouldin": db_jer
    })

    # PAM
    distance_matrix = pairwise_distances(X_array, metric='euclidean')
    np.random.seed(42)
    medoids = np.random.choice(len(X_array), size=k_pam, replace=False).tolist()
    pam = kmedoids(data=distance_matrix, initial_index_medoids=medoids, data_type='distance_matrix')
    pam.process()
    clusters = pam.get_clusters()
    etiquetas_pam = np.zeros(len(X_array), dtype=int)
    for cluster_id, indices in enumerate(clusters):
        for idx in indices:
            etiquetas_pam[idx] = cluster_id
    silhouette_pam = silhouette_score(distance_matrix, etiquetas_pam, metric='precomputed')
    resultados.append({
        "Modelo": f"PAM (k={k_pam})",
        "Silhouette Score": silhouette_pam,
        "Calinski-Harabasz": np.nan,
        "Davies-Bouldin": np.nan
    })

    # Convertir a DataFrame
    df_resultados = pd.DataFrame(resultados).round(4)

    # Gráfico de barras Silhouette
    fig_sil = px.bar(
        df_resultados,
        x="Modelo", y="Silhouette Score",
        text="Silhouette Score",
        title="Comparación de Modelos por Silhouette Score",
        color="Modelo",
        color_discrete_sequence=px.colors.qualitative.Set2
    )
    fig_sil.update_traces(texttemplate='%{text:.4f}', textposition='outside')
    fig_sil.update_layout(yaxis=dict(range=[0, 1]), uniformtext_minsize=8, uniformtext_mode='hide')

    # Tabla de métricas
    tabla = dash_table.DataTable(
        columns=[{"name": col, "id": col} for col in df_resultados.columns],
        data=df_resultados.to_dict("records"),
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'center'},
        style_header={'fontWeight': 'bold', 'backgroundColor': '#f1f1f1'}
    )

    return [
        html.Div([
            dcc.Graph(figure=fig_sil),
            html.H3("Métricas Comparativas", style={"marginTop": "20px"}),
            tabla
        ])
    ]

def generar_pam_pca(X, df_clientes):
    X_array = X.to_numpy()
    distance_matrix = pairwise_distances(X_array, metric='euclidean')
    np.random.seed(42)
    initial_medoids = np.random.choice(len(X_array), size=3, replace=False).tolist()
    pam_instance = kmedoids(data=distance_matrix, initial_index_medoids=initial_medoids, data_type='distance_matrix')
    pam_instance.process()
    clusters = pam_instance.get_clusters()

    labels_pam = np.zeros(len(X_array), dtype=int)
    for cluster_id, indices in enumerate(clusters):
        for index in indices:
            labels_pam[index] = cluster_id

    # PCA para visualización
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_array)
    df_pca_pam = pd.DataFrame(X_pca, columns=["PC1", "PC2"])
    df_pca_pam["cluster_pam"] = labels_pam
    df_pca_pam["cliente"] = df_clientes["cliente"]

    fig = px.scatter(
        df_pca_pam, x="PC1", y="PC2", color="cluster_pam",
        hover_data=["cliente"], title="Distribución de Clústeres PAM con PCA",
        color_continuous_scale="Sunset"
    )
    return fig

def generar_tabla_resumen_pam(X, df_clientes):
    X_array = X.to_numpy()
    distance_matrix = pairwise_distances(X_array, metric='euclidean')
    np.random.seed(42)
    initial_medoids = np.random.choice(len(X_array), size=3, replace=False).tolist()
    pam_instance = kmedoids(data=distance_matrix, initial_index_medoids=initial_medoids, data_type='distance_matrix')
    pam_instance.process()
    clusters = pam_instance.get_clusters()

    labels_pam = np.zeros(len(X_array), dtype=int)
    for cluster_id, indices in enumerate(clusters):
        for index in indices:
            labels_pam[index] = cluster_id

    df_resultado_pam = df_clientes.copy()
    df_resultado_pam["cluster_pam"] = labels_pam

    resumen = df_resultado_pam.groupby("cluster_pam").agg({
        'total_gastado': 'mean',
        'cantidad_total': 'mean',
        'total_facturas': 'mean',
        'num_productos_distintos': 'mean',
        'num_categorias_distintas': 'mean'
    }).round(2).reset_index()

    return dash_table.DataTable(
        columns=[{"name": col, "id": col} for col in resumen.columns],
        data=resumen.to_dict("records"),
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'center'},
        style_header={'fontWeight': 'bold', 'backgroundColor': '#e1e1e1'}
    )

def preparar_datos_modelo(frecuencia_minima):
    df_temp = df[df["cliente"] != "000022222222"].copy()
    meses_dict = {m: i for i, m in enumerate(
        ["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio",
         "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"], start=1)}
    df_temp["mes_num"] = df_temp["mes"].map(meses_dict)
    df_temp["fecha"] = pd.to_datetime(df_temp["anio"] + "-" + df_temp["mes_num"].astype(str).str.zfill(2) + "-01")

    frecuencia = df_temp.groupby("cliente")["fecha"].nunique().reset_index(name="num_meses")
    frecuencia["frecuente"] = (frecuencia["num_meses"] >= frecuencia_minima).astype(int)

    total_compras = df_temp.groupby("cliente")["cantidad"].sum().reset_index(name="total_compras")
    total_gastado = df_temp.groupby("cliente")["valor_total"].sum().reset_index(name="total_gastado")
    num_productos = df_temp.groupby("cliente")["nombre_producto"].nunique().reset_index(name="num_productos")
    num_categorias = df_temp.groupby("cliente")["categoria"].nunique().reset_index(name="num_categorias")
    num_municipios = df_temp.groupby("cliente")["municipio"].nunique().reset_index(name="num_municipios")

    df_modelo = frecuencia.merge(total_compras, on="cliente") \
                          .merge(total_gastado, on="cliente") \
                          .merge(num_productos, on="cliente") \
                          .merge(num_categorias, on="cliente") \
                          .merge(num_municipios, on="cliente")
    
    X = df_modelo[["total_compras", "total_gastado", "num_productos", "num_categorias", "num_municipios"]]
    y = df_modelo["frecuente"]
    return X, y

@app.callback(
    Output('salida_modelo_clasificacion', 'children'),
    Input('slider_frecuencia', 'value')
)
def ejecutar_modelo_clasificacion(frecuencia_minima):

    X, y = preparar_datos_modelo(frecuencia_minima)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print("Distribución de clases en y:", y.value_counts())
    
    if y_train.nunique() < 2:
        return html.Div([
            html.H4("Error: No hay clases suficientes para entrenamiento",
                    style={'color': 'red', 'textAlign': 'center'}),
            html.P("Los datos de entrenamiento solo contienen una clase. Por favor revisa los filtros o datos.")
        ])
        
    modelo = LogisticRegression(max_iter=1000)
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    reporte = classification_report(y_test, y_pred, output_dict=True)

    # ---- Gráfico matriz de confusión ----
    fig = go.Figure(data=go.Heatmap(
        z=cm,
        x=["No Frecuente", "Frecuente"],
        y=["No Frecuente", "Frecuente"],
        colorscale="Blues",
        showscale=False,
        text=cm,
        texttemplate="%{text}"
    ))
    fig.update_layout(title="Matriz de Confusión", xaxis_title="Predicción", yaxis_title="Real")

    # ---- Tabla de métricas ----
    reporte_df = pd.DataFrame(reporte).T.reset_index().rename(columns={"index": "Clase"})
    tabla_reporte = dash_table.DataTable(
        columns=[{"name": col, "id": col} for col in reporte_df.columns],
        data=reporte_df.round(3).to_dict("records"),
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'center'},
        style_header={'backgroundColor': 'lightgray', 'fontWeight': 'bold'}
    )

    # ---- Tabla coeficientes ----
    coef = pd.DataFrame({
        "Variable": X.columns,
        "Coeficiente": modelo.coef_[0]
    }).sort_values(by="Coeficiente", key=abs, ascending=False)
    tabla_coef = dash_table.DataTable(
        columns=[{"name": col, "id": col} for col in coef.columns],
        data=coef.round(3).to_dict("records"),
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'center'},
        style_header={'backgroundColor': 'lightgray', 'fontWeight': 'bold'}
    )

    # ---- Validación cruzada ----
    global scores_log_cv, auc_log_global
    y_log_prob = modelo.predict_proba(X_test)[:, 1]
    scores_log_cv = cross_val_score(modelo, X, y, cv=5)
    auc_log_global = roc_auc_score(y_test, y_log_prob)

    fig_cv = go.Figure()
    fig_cv.add_trace(go.Indicator(
        mode="number",
        value=scores_log_cv.mean(),
        number={"valueformat": ".3f"},
        title={"text": "Precisión CV (5 folds)"}
    ))
    fig_cv.update_layout(height=200)
    
    return [
        html.H3("Matriz de Confusión y Validación Cruzada", style={"marginTop": "20px"}),
        
        html.Div([
            html.Div([dcc.Graph(figure=fig)], style={'width': '65%', 'display': 'inline-block', 'verticalAlign': 'top'}),
            html.Div([dcc.Graph(figure=fig_cv)], style={'width': '35%', 'display': 'inline-block', 'paddingLeft': '20px'})
        ], style={'display': 'flex', 'flexDirection': 'row'}),
        
        html.H3("Reporte de Clasificación", style={"marginTop": "40px"}),
        tabla_reporte,

        html.H3("Coeficientes del Modelo", style={"marginTop": "40px"}),
        tabla_coef
    ]

@app.callback(
    Output('salida_modelo_arbol', 'children'),
    Input('slider_frecuencia', 'value')
)
def ejecutar_modelo_arbol(frecuencia_minima):
    from sklearn.tree import DecisionTreeClassifier, plot_tree

    X, y = preparar_datos_modelo(frecuencia_minima)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    if y_train.nunique() < 2:
        return html.Div("No hay suficientes clases para entrenar el árbol de decisión.")

    # ---- Modelo Árbol de Decisión ----
    tree_model = DecisionTreeClassifier(min_samples_leaf=3, max_depth=5)
    tree_model.fit(X_train, y_train)
    y_pred_tree = tree_model.predict(X_test)
    cm_tree = confusion_matrix(y_test, y_pred_tree)
    reporte = classification_report(y_test, y_pred_tree, output_dict=True)
    
    # Tabla de métricas (clasification_report)
    reporte_df = pd.DataFrame(reporte).T.reset_index().rename(columns={"index": "Clase"})
    tabla_reporte = dash_table.DataTable(
        columns=[{"name": col, "id": col} for col in reporte_df.columns],
        data=reporte_df.round(3).to_dict("records"),
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'center'},
        style_header={'backgroundColor': 'lightgray', 'fontWeight': 'bold'}
    )

    # ---- Gráfico Matriz de Confusión ----
    fig_tree = go.Figure(data=go.Heatmap(
        z=cm_tree,
        x=["No Frecuente", "Frecuente"],
        y=["No Frecuente", "Frecuente"],
        colorscale="Greens",
        showscale=False,
        text=cm_tree,
        texttemplate="%{text}"
    ))
    fig_tree.update_layout(title="Matriz de Confusión - Árbol de Decisión", xaxis_title="Predicción", yaxis_title="Real")

    # ---- Validación Cruzada ----
    global scores_tree_cv, auc_tree_global
    scores_tree_cv = cross_val_score(tree_model, X, y, cv=5)
    y_tree_prob = tree_model.predict_proba(X_test)[:, 1]
    auc_tree_global = roc_auc_score(y_test, y_tree_prob)

    fig_cv_tree = go.Figure()
    fig_cv_tree.add_trace(go.Indicator(
        mode="number",
        value=scores_tree_cv.mean(),
        number={"valueformat": ".3f"},
        title={"text": "Precisión CV (5 folds)"}
    ))
    fig_cv_tree.update_layout(height=200)

    buf = io.BytesIO()
    plt.figure(figsize=(18, 8))
    plot_tree(
        tree_model,
        feature_names=X.columns,
        class_names=["No Frecuente", "Frecuente"],
        filled=True,
        rounded=True,
        fontsize=10
    )
    plt.tight_layout()
    plt.savefig(buf, format="png")
    plt.close()
    buf.seek(0)
    img_base64 = base64.b64encode(buf.read()).decode("utf-8")
    img_html = html.Img(src=f"data:image/png;base64,{img_base64}", style={"width": "100%"})
    
    return [
        html.H3("Matriz de Confusión y Validación Cruzada", style={"marginTop": "20px"}),
        
        html.Div([
            html.Div([dcc.Graph(figure=fig_tree)], style={
                'width': '65%',
                'display': 'inline-block',
                'verticalAlign': 'top'
            }),
            html.Div([dcc.Graph(figure=fig_cv_tree)], style={
                'width': '35%',
                'display': 'inline-block',
                'paddingLeft': '20px'
            })
        ], style={'display': 'flex', 'flexDirection': 'row'}),
        
        html.H3("Reporte de Clasificación", style={"marginTop": "40px"}),
        tabla_reporte,

        html.H3("Visualización del Árbol de Decisión", style={"marginTop": "40px"}),
        img_html,
        html.P("* La carga del árbol puede tardar unos segundos. Si no se visualiza correctamente, por favor recarga la página",
            style={
                'textAlign': 'justify',
                'fontSize': '16px',
                'marginTop': '10px',
                'marginBottom': '20px',
                'paddingLeft': '20px',
                'paddingRight': '20px',
                'fontFamily': 'sans-serif',
                'color': '#333'
            }
        )
    ]

@app.callback(
    Output('comparacion_modelos_clasificacion', 'children'),
    Input('slider_frecuencia', 'value')
)
def comparar_modelos_clasificacion(frecuencia_minima):
    X, y = preparar_datos_modelo(frecuencia_minima)

    if y.nunique() < 2:
        return html.Div("No hay clases suficientes para comparar modelos.")

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    log_model = LogisticRegression(max_iter=1000).fit(X_train, y_train)
    tree_model = DecisionTreeClassifier(max_depth=5, min_samples_leaf=3).fit(X_train, y_train)

    y_log_prob = log_model.predict_proba(X_test)[:, 1]
    y_tree_prob = tree_model.predict_proba(X_test)[:, 1]

    fpr_log, tpr_log, _ = roc_curve(y_test, y_log_prob)
    fpr_tree, tpr_tree, _ = roc_curve(y_test, y_tree_prob)
    auc_log = roc_auc_score(y_test, y_log_prob)
    auc_tree = roc_auc_score(y_test, y_tree_prob)

    scores_log_cv = cross_val_score(log_model, X, y, cv=5)
    scores_tree_cv = cross_val_score(tree_model, X, y, cv=5)

    resumen = pd.DataFrame({
        "Modelo": ["Regresión Logística", "Árbol de Decisión"],
        "Precisión Test": [log_model.score(X_test, y_test), tree_model.score(X_test, y_test)],
        "Precisión CV": [scores_log_cv.mean(), scores_tree_cv.mean()],
        "AUC": [auc_log, auc_tree]
    }).round(3)

    fig_roc = go.Figure()
    fig_roc.add_trace(go.Scatter(x=fpr_log, y=tpr_log, mode='lines', name=f"Logística (AUC={auc_log:.2f})"))
    fig_roc.add_trace(go.Scatter(x=fpr_tree, y=tpr_tree, mode='lines', name=f"Árbol (AUC={auc_tree:.2f})"))
    fig_roc.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', line=dict(dash='dash'), name='Aleatorio'))
    fig_roc.update_layout(title="Curvas ROC", xaxis_title="False Positive Rate", yaxis_title="True Positive Rate")

    fig_barras = px.bar(
        resumen, x="Modelo", y="Precisión CV", color="Modelo",
        text="Precisión CV", title="Precisión media (5-fold CV)",
        color_discrete_sequence=["royalblue", "seagreen"]
    )
    fig_barras.update_traces(texttemplate='%{text:.3f}', textposition='outside')
    fig_barras.update_layout(yaxis=dict(range=[0, 1]))

    tabla = dash_table.DataTable(
        columns=[{"name": col, "id": col} for col in resumen.columns],
        data=resumen.to_dict("records"),
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'center'},
        style_header={'fontWeight': 'bold', 'backgroundColor': '#f8f8f8'}
    )

    return [
        html.H3("Curvas ROC", style={'marginTop': '30px'}),
        dcc.Graph(figure=fig_roc),
        html.H3("Precisión CV comparativa", style={'marginTop': '30px'}),
        dcc.Graph(figure=fig_barras),
        html.H3("Resumen de métricas", style={'marginTop': '30px'}),
        tabla
    ]


# -------------------------------
# 6. Ejecutar la app
# -------------------------------
if __name__ == '__main__':
    app.run(debug=True)

In [3]:
# Para abrir en una pagina y tener mayor visualización, se recomienda abrir este link 
http://127.0.0.1:8050/

SyntaxError: invalid syntax (4173231028.py, line 2)

In [ ]:
logo_src = "..."